# Data Science Pandas - Joins, Time Series & Rolling Statistics

## Tasks Today:

1) <b>Pandas</b> <br>
 &nbsp;&nbsp;&nbsp;&nbsp; a) Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - merge() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Breaking Down the Inner Join <br>
 &nbsp;&nbsp;&nbsp;&nbsp; b) Outer Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Use Case Example <br>
 &nbsp;&nbsp;&nbsp;&nbsp; c) Rolling Statistics and Time Series w/Pandas <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - read_csv() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - shift() <br>

In [1]:
import numpy as np
import pandas as pd

### Joins <br>
<p>Very often you will have to combine data from several different sources to obtain the actual dataset you need for your exploration or modeling. Pandas draws heavily on SQL in its API design for joins, and if you are familiar with SQL joins, then Pandas will come naturally. Imagine you have two tables, one with customer name and gender, and the other with their purchases:</p>

In [2]:
# Generate some fake data
city_data = {
    1: 'Chicago',
    2: 'Boston',
    3: 'Dallas',
    4: 'New York',
    5: 'Portsmouth'
}

np.random.seed(321)

customer_id = np.arange(1000,2000)
ages = np.random.randint(18,65,1000)
city = [ city_data[key] for key in np.random.randint(1, 6, 1000) ]

<p>Let's assume we have a dataset containing the customer ID's cities, and ages for a set of customers. Suppose further, that we have a second table, containing some customer ID's, and a feedback survey rating from thoes customers. Suppose finally, that you as an analyst are asked the following question: What city tends to give the best feedback for our product? We'll need a join operation to get the answer.</p>

In [3]:
# create cust_demographics dataframe
df = pd.DataFrame.from_dict({
    'customer_id' : customer_id,
    'ages': ages,
    'city': city
})

df.head(5)

,customer_id,ages,city
0,1000,44,Chicago
1,1001,49,Boston
2,1002,59,Portsmouth
3,1003,26,Portsmouth
4,1004,35,Dallas


In [5]:
# Generate some other fake data
customer_id = np.random.randint(1000, 3000, 1000)
customer_rating = np.random.randint(1,11,1000)

surveys = pd.DataFrame.from_dict({
    'customer_id': customer_id,
    'customer_rating': customer_rating
})

# drop any duplicate information with same id's
surveys = surveys.drop_duplicates(subset='customer_id')

surveys.head(5)

,customer_id,customer_rating
0,2047,6
1,2471,6
2,1882,7
3,1387,10
4,1912,7


##### merge()

<p>We can zip the two dataframes up with an inner join, using the Pandas API as follows:</p>

In [6]:
ratings = df.merge(surveys, on='customer_id', how='inner')

ratings.head(10)

,customer_id,ages,city,customer_rating
0,1002,59,Portsmouth,6
1,1004,35,Dallas,5
2,1005,58,New York,10
3,1009,26,Boston,3
4,1013,45,New York,5
5,1014,44,Chicago,4
6,1015,31,Dallas,5
7,1017,49,Chicago,6
8,1019,55,Portsmouth,3
9,1020,52,Chicago,1


<p>Now it's a simple groupby operation to reveal which city has the higher rating:</p>

In [7]:
ratings.groupby('city').mean().head(10)

,customer_id,ages,customer_rating
city,,,
Boston,1465.421053,39.355263,5.355263
Chicago,1531.924731,40.311828,5.548387
Dallas,1518.600000,41.537500,5.450000
New York,1503.260274,40.890411,5.465753
Portsmouth,1525.402439,41.146341,5.682927


##### Breaking Down the Inner Join <br>
<p>Let's take a closer look at the actual mechanics here however. There is some subtlety and nuance that is best understood by thinking of an inner join as two steps: a cartesian product, and then a filter. Observe the result of the following inner join:</p>

In [19]:
d = {
    'A': [1, 2, 3, 4],
    'B': ['a', 'b', 'c', 'd']
}

df1 = pd.DataFrame.from_dict(d)

df1.head()

,A,B
0,1,a
1,2,b
2,3,c
3,4,d


In [20]:
d = {
    'A': [1, 2, 6, 2],
    'C': ['e', 'f', 'g', 'h']
}

df2 = pd.DataFrame.from_dict(d)

df2.head()

,A,C
0,1,e
1,2,f
2,6,g
3,2,h


In [21]:
df1.merge(df2, on='A', how='inner').head()

,A,B,C
0,1,a,e
1,2,b,f
2,2,b,h


<p>Somehow, we ended up with more rows in the result than either of the original tables! This is because we joined on a $\textit{non-unique index}$. The join starts by computing all the possible combinations of rows, and then filters them based on the condition. Because of the duplicates, multiple combinations of rows make it through!</p>

### Outer Joins <br>
<p>An $\textit{outer join}$ produces all of the rows from the left table, and joins whatever rows it can to the right table, filling anywhere it cant with NaN's or None values. Below is a brief example, and then we will discuss a possible use case for this type of join.</p>

In [22]:
df1 = pd.DataFrame.from_dict({'A': [1, 2, 3], 'B': ['a', 'b', 'c']})
df2 = pd.DataFrame.from_dict({'A': [1], 'C': ['d']})

In [23]:
df1.head()

,A,B
0,1,a
1,2,b
2,3,c


In [24]:
df2.head()

,A,C
0,1,d


In [25]:
df1.merge(df2, on='A', how='outer')

# Throw an error if column names are same name so 
# if df2 had 'a' and 'b' would do (l_suffix='left', r_suffix='right')

,A,B,C
0,1,a,d
1,2,b,NaN
2,3,c,NaN


##### Use Case Example

<p>As an example use case, let's imagine we have again the demographics and survey response tables from above:</p>

In [9]:
df.head(5)

,customer_id,ages,city
0,1000,44,Chicago
1,1001,49,Boston
2,1002,59,Portsmouth
3,1003,26,Portsmouth
4,1004,35,Dallas


<p>Now, you are asked as an analyst to compare the demographics of survey responders to the demographics of all your customers combined.</p>

In [11]:
ratings = df.merge(surveys, on='customer_id', how='outer')

ratings['customer_rating'] = ratings['customer_rating'].fillna('N/A') # replace NaN with N/A

ratings.head(10)

,customer_id,ages,city,customer_rating
0,1000,44.0,Chicago,N/A
1,1001,49.0,Boston,N/A
2,1002,59.0,Portsmouth,6
3,1003,26.0,Portsmouth,N/A
4,1004,35.0,Dallas,5
5,1005,58.0,New York,10
6,1006,44.0,Boston,N/A
7,1007,42.0,Chicago,N/A
8,1008,55.0,Chicago,N/A
9,1009,26.0,Boston,3


In [15]:
ratings.groupby(['city', 'customer_rating']).size()

city        customer_rating
Boston      1.0                  6
            2.0                  8
            3.0                  9
            4.0                  9
            5.0                  7
            6.0                  8
            7.0                  9
            8.0                  8
            9.0                  8
            10.0                 4
            N/A                110
Chicago     1.0                 10
            2.0                  8
            3.0                  6
            4.0                  8
            5.0                 11
            6.0                 10
            7.0                 12
            8.0                 16
            9.0                  7
            10.0                 5
            N/A                135
Dallas      1.0                  7
            2.0                  7
            3.0                  8
            4.0                 11
            5.0                 12
            6.0            

In [19]:
# average age of people that answered the survey
answered = ratings[ratings['customer_rating'] != 'N/A']

avg_answered = answered['ages'].mean()

avg_answered

40.648514851485146

In [20]:
# find the average age of people that did not answer the survey
answered = ratings[ratings['customer_rating'] == 'N/A']

avg_answered = answered['ages'].mean()

avg_answered

40.65436241610738

### Rolling Statistics and Time Series with Pandas

##### read_csv()

In [22]:
weather = pd.read_csv('files/chiweather.csv', sep=';')

In [23]:
weather.head(10)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd]
0,2017,11,25,0,0,51.85,0.0,17.31,327.13
1,2017,11,25,1,0,49.14,0.0,20.53,330.64
2,2017,11,25,2,0,46.42,0.0,21.41,327.80
3,2017,11,25,3,0,44.13,0.0,21.79,324.19
4,2017,11,25,4,0,42.22,0.0,21.03,321.91
5,2017,11,25,5,0,40.75,0.0,19.62,316.85
6,2017,11,25,6,0,40.71,0.0,19.17,318.31
7,2017,11,25,7,0,39.22,0.0,18.51,313.53
8,2017,11,25,8,0,39.49,0.0,18.73,310.16
9,2017,11,25,9,0,40.73,0.0,19.80,311.79


In [26]:
# reset index after grouping
weather = weather.groupby(['Year', 'Month', 'Day']).mean().reset_index()

In [27]:
weather.head(10)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd]
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500


##### shift() <br>
<p>Shift index by desired number of periods with an optional time freq</p>

In [33]:
# print(weather.keys())
weather['last_temp'] = weather['Temperature  [2 m above gnd]'].shift(1)

weather.head(10)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd],last_temp
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667,NaN
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750,41.208750
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083,38.298333
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333,45.611250
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000,48.497500
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000,40.122917
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583,43.618750
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500,41.860417


In [42]:
# create a new column that takes the difference in temperature from the previous day as an absolute value
# call column 'dif_last_day'
# print(weather.keys())

weather['dif_last_day'] = abs(weather['Temperature  [2 m above gnd]'] - weather['last_temp'])

weather.head(10)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd],last_temp,dif_last_day
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667,41.208750,0.0
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750,38.298333,0.0
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083,45.611250,0.0
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333,48.497500,0.0
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000,40.122917,0.0
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000,43.618750,0.0
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583,41.860417,0.0
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500,45.324167,0.0
